# Country - Web Scraping

>- It's necessary a premium login to extract Segments Information, a free account doesn't have full access to all data.
>- For this script you can use either a list of logins, or setup a unique login through the **Function** `change_user ()`, with `number_logins=1` and any login of your choice
>- For a web scraping process is important to have a exception when the Internet connection is lost, for this was created the **Function** `connect()` that tests the connection before each Action, and a loop while in each Action too to garantee to scrap all the information
>- Allow to download gpx file, you should create a folder to the **file1** in the **download_dir**

The last update was at February, 28, 2021.

<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Environment" data-toc-modified-id="Environment-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Environment</a></span></li></ul></li><li><span><a href="#Steps-functions" data-toc-modified-id="Steps-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Steps functions</a></span></li><li><span><a href="#Import-activities-data" data-toc-modified-id="Import-activities-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Import activities data</a></span></li><li><span><a href="#Create-bookmarklet" data-toc-modified-id="Create-bookmarklet-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Create bookmarklet</a></span></li><li><span><a href="#Create-session,-open-page-and-change-language-to-English" data-toc-modified-id="Create-session,-open-page-and-change-language-to-English-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Create session, open page and change language to English</a></span></li><li><span><a href="#Download-GPX-files" data-toc-modified-id="Download-GPX-files-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Download GPX files</a></span></li></ul></div>

## Setup

In [ ]:
import time
import datetime
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pyversions  # https://pypi.org/project/pyversions/
import sys, os
import urllib.request
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import sys, os
import geopy
from geopy.geocoders import Nominatim
import gpxpy
import shutil
import urllib
pyversions.versions();

### Environment

In [ ]:
#parameters
file = 'file1'
# specify custom download directory, you should create a folder with the same name as the file in directory bellow. 
#For example: C:\file1
download_dir = r"C:"
#It´s used a group of logins to access the Strava plataform and scraping the information
#id_login it´s the first index to rotate logins
id_login=0 #choose the first index in the logins list to rotate the accesses
number_logins = 1 #choose the number of logins to use
first_athlete=0 #variable to indicate the first position in the athletes list
last_athlete=1 #variable to indicate the last position in the athletes list
import_strava ="0-1.csv" #input list with all the athletes to scrap details informations

In [ ]:
download_dir = download_dir + '\\'+ file
pd.set_option('display.float_format', lambda x: '%.4g' % x)
options = webdriver.ChromeOptions()
prefs = {"download.default_directory":download_dir+"\\","directory_upgrade": True}
options.add_experimental_option("prefs", prefs)

path2 = r'../Datasets'
logins=pd.read_excel(os.path.join(path2,'logins.xlsx'))

count_logins = 1
aux_id = id_login #variable used to compare the current position of the index with the initial
export = file+'.csv'

gpx_country = []
activity_id=[]
athlete_id=[]
#number of registers of countries, there is a condition, if number_info_country >= 10, stops saving the information,
#and pass to another athlete
number_info_country = 0

In [ ]:
logins.columns

## Steps functions

Bellow the list of functions with their descriptions:

- **open_section** - create a new instance for selenium webdriver, and load the login page
- **login** - fill down the form and click
- **user_access** - kill the session and starts a new one with other logins, works together with **change_user** function
- **change_user** - this function changes the logged user if you have a list of logins. It's import to avoid 429 error (too many requests)
- **select_english_language** - to scrap all the information in English, this function changes the language view to English
- **go_link** - It's used to load activity page
- **create_dataset** - organize the informations and save a **.csv** file
- **connect** - check the Internet connection, there a loop while to wait the Internet connection
- **page_loaded** - there is a sequence to load the page, and we can check if the activity page was loaded. The script checks if the activity page was loaded through the information of date - It was searched the number "2" from 2000's, and when the activity isn't found, the URL returns to homepage Strava, and we can check if exists the information "following", because this is a information presents only on homepage.

For example: the response shows "Activities for Nov **2**0**2**0". There are two numbers 2, and the page is loaded. For any reason, if the activity doesn't exist anymore, so the website loads the homepage of the user. And we can check too.


- **scraping:** athletes country are stored in this part
- **wait_for_download_and_rename:** for each selected activity the scripts tries to download the [GPX](https://en.wikipedia.org/wiki/GPS_Exchange_Format) file, and renames each downloaded file to a specific name, given by user. This is necessary because the default name is the name of the activity. After downloading the GPX file, It was used the [gpxpy](https://pypi.org/project/gpxpy/) lib to read the gpx file, and find each latitude and longitude position to gather them, and lasting use the first position (we need only one position to find out the athletes country) as an input through [geopy](https://pypi.org/project/geopy/) lib, and waiting for the country name

In [ ]:
from ipynb.fs.defs.webscrap_athletes import login, open_section, change_user, select_english_language,\
    go_link, connect

#https://stackoverflow.com/questions/34548041/selenium-give-file-name-when-downloading/56570364
# function to wait for download to finish and then rename the latest downloaded file
def wait_for_download_and_rename(newFilename):
    global download_dir
    # function to wait for all chrome downloads to finish
    def chrome_downloads(drv):
        if not "chrome://downloads" in drv.current_url: # if 'chrome downloads' is not current tab
            drv.execute_script("window.open('');") # open a new tab
            drv.switch_to.window(driver.window_handles[1]) # switch to the new tab
            drv.get("chrome://downloads/") # navigate to chrome downloads
        return drv.execute_script("""
            return document.querySelector('downloads-manager')
            .shadowRoot.querySelector('#downloadsList')
            .items.filter(e => e.state === 'COMPLETE')
            .map(e => e.filePath || e.file_path || e.fileUrl || e.file_url);
            """)
    # wait for all the downloads to be completed
    dld_file_paths = WebDriverWait(driver, 120, 1).until(chrome_downloads) # returns list of downloaded file paths
    # Close the current tab (chrome downloads)
    if "chrome://downloads" in driver.current_url:
        driver.close()
    # Switch back to original tab
    driver.switch_to.window(driver.window_handles[0]) 
    # get latest downloaded file name and path
    dlFilename = dld_file_paths[0] # latest downloaded file from the list
    # wait till downloaded file appears in download directory
    time_to_wait = 20 # adjust timeout as per your needs
    time_counter = 0
    while not os.path.isfile(dlFilename):
        time.sleep(1)
        time_counter += 1
        if time_counter > time_to_wait:
            break
    # rename the downloaded file
    shutil.move(dlFilename, os.path.join(download_dir,newFilename))
    return

def page_loaded(driver,url):
    
    print('Page loaded?')
    CRED = '\033[92m'
    CEND = '\033[0m'
    while True:
        connect()
        try:
            #verify the activity page was loaded through the information of date - It was searched the number "2" from 2000's
            loading_page = driver.find_element_by_xpath('//*[@id="heading"]/div/div/div[1]/div/div/time')
            loading_page_html=loading_page.get_attribute('innerHTML')
            loading_page_soup=BeautifulSoup(loading_page_html, "html.parser")

            if re.search('2',str(loading_page_soup),re.IGNORECASE):
                print(CRED+'OKAY !!!!'+CEND)
                break

        except:
            # Loading page
            #when the activity isn´t found, the url returns to homepage Strava, and we can check with the information "following"
            try:    
                loading_page = driver.find_element_by_xpath('//*[@id="athlete-profile"]/div[1]/ul/li[1]/a/div')
                loading_page_html=loading_page.get_attribute('innerHTML')
                loading_page_soup=BeautifulSoup(loading_page_html, "html.parser")

                if re.search('Following',str(loading_page_soup),re.IGNORECASE):
                    print(CRED+'OKAY !!!! However there is not GPX file at the page'+CEND)
                    download = False
                    return download
                    break
                else:
                    go_link(driver,url)
                    time.sleep(5)
                    continue
            
            except: 
                go_link(driver,url)
                time.sleep(5)
                continue
                
def download_gpx(driver,url,activity):
    while True:
        connect()
        try:
            #check if the activity page was loaded through the information of date - It was searched the number "2" from 2000's
            loading_page = driver.find_element_by_xpath('//*[@id="selected-map"]')
            loading_page_html=loading_page.get_attribute('innerHTML')
            loading_page_soup=BeautifulSoup(loading_page_html, "html.parser")
            
            #check if the map loaded trough the "Standard map" button
            if re.search('standard',str(loading_page_soup),re.IGNORECASE):
                print('Map loaded!!')
                
                #call bookmarklet to allow any user to catch the gpx file
                driver.execute_script(bookmarkletCode);
                time.sleep(5)
                element=driver.find_element_by_xpath('//*[@id="mapstogpxstrava_popup_creategpx"]')
                driver.execute_script("arguments[0].click();", element) #click
                # call the function to wait for download to finish and rename the downloaded file
                wait_for_download_and_rename(file+'.gpx')
                # Parsing an existing file:
                #gpx_file = open('D:/Dropbox/Mestrado/Estudos/Bases de estudo/Web Scraping/webscrap strava atividade/Notebooks/file1/file1.gpx', 'r',encoding='utf-8')
                
                gpx_file = open(re.sub('\\\\','/',download_dir)+'/'+file+'.gpx', 'r',encoding='utf-8')
                gpx = gpxpy.parse(gpx_file)
                coordinates = str(gpx.tracks[0].segments[0].points[0].latitude)+', '+ str(gpx.tracks[0].segments[0].points[0].longitude)

                locator = Nominatim(user_agent="myGeocoder")
                location = locator.reverse(coordinates)

                gpx_country.append(location.raw['address']['country'])
                
                activity_id.append(df_activity['activity_id'][activity])
                athlete_id.append(df_activity['athlete_id'][activity])
                download = True
                
            else:
                download = False
            #variable download is used to flag if download happend or not
            return download
            break
                
        except:
            go_link(driver,url)
            time.sleep(5)
            continue
            
def create_dataset():
    df_activity_id=pd.DataFrame(activity_id,columns=['activity_id'])
    df_athlete_id=pd.DataFrame(athlete_id,columns=['athlete_id'])
    df_gpx_country=pd.DataFrame(gpx_country,columns=['country'])
    
    df=pd.concat([df_athlete_id,df_activity_id,df_gpx_country],axis=1)
    
    df.to_csv(export,index=False,sep=';')
    print('Dataset created')
    
def user_access(driver,url):
    global count_logins
    global aux_id
    
    if count_logins%3==0 and number_logins>1: #check if there is more than 1 login and the number of logins up to the limit for another
        
        connect()
        driver.close()
        driver.quit()
        
        aux_id = aux_id + 1
        if (aux_id - id_login)>=number_logins:
            aux_id = id_login
        
        driver = open_section()
        email, key = change_user(aux_id,logins)
        login(driver,email, key)
        select_english_language(driver,aux_id)
        go_link(driver,url=url)
        
    
    count_logins=count_logins+1
    return driver

## Import activities data

In [ ]:
athletes_list = pd.read_csv(os.path.join(path2,import_strava),sep=';',nrows=1)
athletes_list.columns

In [ ]:
usecols = [2,7]
athletes_list = pd.read_csv(os.path.join(path2,import_strava),sep=';',
                            usecols=usecols,
                            encoding='utf-8', verbose=False)
athletes_list

In [ ]:
#resample the values, to shuffle the activities
athletes_list = athletes_list.sample(frac=1,random_state=1).reset_index(drop=True)
unique = athletes_list.drop_duplicates(subset='athlete_id', keep='first').reset_index(drop=True)
unique.head()

## Create bookmarklet

It was created a bookmarklet ([see](https://mapstogpx.com/strava/) its doc) through a javascript code. And allows to access and download the [gpx](https://en.wikipedia.org/wiki/GPS_Exchange_Format) file.

In [ ]:
#https://mapstogpx.com/strava/
#This code allows to use any free user to catch the gpx file
bookmarkletURL = "javascript:(function()%7Bfunction%20callback()%7B(function(%24)%7Bvar%20jQuery%3D%24%3Bfunction%20callback()%7BgetGpx()%7Dvar%20s%3Ddocument.createElement(%22script%22)%3Bs.src%3D%22https%3A%2F%2Fmapstogpx.com%2Fstrava%2Fmapstogpxstrava.js%22%3Bif(s.addEventListener)%7Bs.addEventListener(%22load%22%2Ccallback%2Cfalse)%7Delse%20if(s.readyState)%7Bs.onreadystatechange%3Dcallback%7Ddocument.body.appendChild(s)%3B%7D)(jQuery.noConflict(true))%7Dvar%20s%3Ddocument.createElement(%22script%22)%3Bs.src%3D%22https%3A%2F%2Fmapstogpx.com%2Fstrava%2Fjquery.min.js%22%3Bif(s.addEventListener)%7Bs.addEventListener(%22load%22%2Ccallback%2Cfalse)%7Delse%20if(s.readyState)%7Bs.onreadystatechange%3Dcallback%7Ddocument.body.appendChild(s)%3B%7D)()"
bookmarkletCode = bookmarkletURL.split("javascript:")[1]
bookmarkletCode = urllib.parse.unquote(bookmarkletCode)

## Create session, open page and change language to English

In [ ]:
driver = open_section()
email, key = change_user(id_login,logins)
login(driver,email, key)
select_english_language(driver,id_login)

## Download GPX files

In [ ]:
for athlete in range(first_athlete,last_athlete):

    #for each athlete is selected their activities
    df_activity = athletes_list[athletes_list['athlete_id']==unique['athlete_id'][athlete]]
    df_activity.reset_index(drop=True,inplace=True)

    for activity in range(df_activity.shape[0]):

        CRED = '\033[1m'
        CEND = '\033[0m'
        print(CRED + 'Athlete - ' +str(athlete)+ " out of " + str(last_athlete) + " | Activity - " + str(activity) + " out of " + str(df_activity.shape[0])+ CEND)
        
        url = df_activity['activity_id'][activity]
        go_link(driver,url=url)
        driver = user_access(driver,url)
        
        if(page_loaded(driver,url) == False):
            continue
            
        if(download_gpx(driver,url,activity)==False):
            continue
            
        number_info_country += 1
        CRED = '\033[94m'
        print(CRED + str(number_info_country) + ' out of 10'+CEND)
        if number_info_country == 10:
            number_info_country=0
            break
    
    create_dataset()
    